# Data validation with ORMs

LaminDB implements data validation at the ORM level by fully integrating the SQLModel ORM with pydantic type checking.

Let's take a look at data validation behavior in LaminDB.

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import pytest
from pydantic import ValidationError

## Missing Field Error

Let's create a `User` instance without the required `email` and `handle` fields.

In [ ]:
with pytest.raises(ValidationError) as e:
    exception = e
    user_missing = lns.User(id="123")
print(exception.exconly())

## Simple Type Error

Let's create a `Transform` instance with the wrong type for the optional field `name`.

In [ ]:
from datetime import datetime

with pytest.raises(ValidationError) as e:
    exception = e
    invalid_transform = ln.Transform(name=datetime.now())
print(exception.exconly())

## Simple Categorical Error

Let's pass an invalid categorical to the `type` field in `Usage`, which only accepts the values 'ingest', 'insert', 'select', 'update', 'delete', 'load', and 'link'.

In [ ]:
from lnschema_core._core import SQLModel
from lnschema_core.dev.type import usage as event_type
from sqlmodel import Field

In [ ]:
class Usage(SQLModel, table=True):  # type: ignore
    id: str = Field(default=None, primary_key=True)
    type: event_type = Field(nullable=False, index=True)

In [ ]:
assert Usage(type="ingest")

In [ ]:
with pytest.raises(ValidationError) as e:
    exception = e
    invalid_usage = Usage(type="invalid categorical")
print(exception.exconly())

## Special Cases in Data Validation

Data validation with the LaminDB ORM mirrors the standard Pydantic behavior, including variable casting (see example #1 below) and extra field behaviors (see example #2 below). These can be changed through Pydantic's configuration.

The only difference in behavior between LaminDB and Pydantic is strict type checking for `Relationship` fields (see example #3 below), which is implemented in LaminDB.

## Special Cases in Data Validation #1: Variable Casting

LaminDB mirrors Pydantic's default behavior of casting input variables to conform to field types (see details in [Pydantic's documentation](https://docs.pydantic.dev/usage/models/#data-conversion)).

Let's take a look at the default behavior by creating transform instances with `int` and `bool` inputs to the `name` field, which is string-typed in the schema.

In [ ]:
# Name (int) is cast to str
transform_name_int_to_str = ln.Transform(name=1)
type(transform_name_int_to_str.name)

In [ ]:
# Name (bool) is cast to str
transform_name_bool_to_str = ln.Transform(name=True)
type(transform_name_int_to_str.name)

## Special Cases in Data Validation #2: Extra Fields

LaminDB also mirror's Pydantic default behavior of accepting extra fields not defined in the schema.

In [ ]:
# No error is raised for the extra field
transform = ln.Transform(name="Test", extra_field="This field is not defined in the schema")

## Special Cases in Data Validation #3: Strict Type Checking for Relationship

Differently from Pydantic, LaminDB enforces strict type checking for `Relationship` fields.

Below is a simple example of Pydantic's lenient type checking for `Relationship` fields. Rather than enforcing the `Car` type in the `Wheel.car` field, it only enforces type-checking on the attributes of the input object.

In [ ]:
from sqlmodel import SQLModel, Field, Relationship
from typing import Optional, List


class Car(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

    wheels: List["Wheel"] = Relationship()


class Wheel(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

    car: Optional["Car"] = Relationship()


class Bird(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

In [ ]:
# Pydantic does not raise a validation error for wrong type in the car field
wheel = Wheel(name="Test Wheel", car=Bird(name="Test"))

LaminDB, on the other hand, enforces strict type checking for `Relationships`.

In [ ]:
with pytest.raises(TypeError) as e:
    exception = e
    run = lns.Run(name="Test Run", transform=Bird(name="This is not a Transform"))
print(exception.exconly())

## Appendix

In [ ]:
import nbproject as nb
import pytest
from lnschema_core import File, Transform, Run
from lnschema_core.dev.sqlmodel import SQLModelPrefix
from sqlmodel import Relationship, Field, ForeignKeyConstraint
from typing import Optional
from pydantic import ValidationError

In [ ]:
# non-optional created_at field is correctly ignored (server-side autopopulation)
transform = Transform(name="My test pipeline")

In [ ]:
# invalid type for created_at field is correctly raised
with pytest.raises(ValidationError):
    transform = Transform(created_at="invalid type")

In [ ]:
# relationship validation works correctly (no error raised for valid input)
run = Run(name="Test Run", transform=transform)

In [ ]:
# relationship validation works correctly (ORM functionality is maintained)
assert run.transform

In [ ]:
# relationship validation works correctly (strict type checking, non-model)
with pytest.raises(TypeError):
    run = Run(name="Test Run", transform="invalid transform")

In [ ]:
# relationship validation works correctly (strict type checking, valid model)
with pytest.raises(TypeError):
    file = File(source=Transform(name="My test pipeline"))

In [ ]:
# relationship validation works correctly (strict type checking, resolved ForwardRef)
class FileRel(SQLModelPrefix, table=True):
    id: str = Field(primary_key=True, default=None)
    file_id: Optional[str] = Field(foreign_key="core.file.id", index=True)
    file: Optional["File"] = Relationship()


file = File(name="myfile", source_id="raf", storage_id="g4s")
file_real = FileRel(file=file)

In [ ]:
# auto-population of required fk constraints works correctly
class FileRelFK(SQLModelPrefix, table=True):
    id: str = Field(primary_key=True, default=None)
    file_id: str = Field(foreign_key="core.file.id", index=True)
    file: File = Relationship()


file = File(name="myfile", source_id="raf", storage_id="g4s")
file_rel = FileRelFK(file=file)


# auto-population of required fk constraints works correctly (composite fk)
class TransformRelFK(SQLModelPrefix, table=True):
    __table_args__ = (ForeignKeyConstraint(["transform_id", "transform_v"], ["core.transform.id", "core.transform.v"]),)
    id: str = Field(primary_key=True, default=None)
    transform_id: str = Field(index=True)
    transform_v: str = Field(index=True)
    transform: Transform = Relationship()


transform = Transform(name="mytransform")
transform_rel = TransformRelFK(transform=transform)

In [ ]:
# file validation works correctly (missing field errors are raised even when file is instantiated with custom constructor)
with pytest.raises(ValidationError):
    file = File(name="no data objects")

In [ ]:
# file validation works correctly (no errors raised when file is conformantly instantiated with custom constructor)
file = File(name="myfile", source_id=nb.meta.store.id, storage_id="123")